In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import vectorbtpro as vbt

from vctr.data.data_loader import get_data

vbt.settings.set_theme('dark')

In [15]:
from datetime import datetime

data = get_data('NEAR', '1h')

In [26]:
from vctr.data.labeling import label_data_extrema_multi, label_data_trends_binary

data2 = label_data_trends_binary(data, .0150)

data2.label.value_counts(normalize=True)

0.0    0.509163
1.0    0.490837
Name: label, dtype: float64

In [7]:
import plotly.graph_objs as go


def plot_price_with_labels(data):
    green_bars = data[data.label == 1]['close']
    red_bars = data[data.label == 2]['close']

    # Create line plot for close prices
    price_trace = go.Scatter(x=data.index, y=data['close'], mode='lines', name='Close Price', line=dict(width=1))

    # Create green and red vertical bar shapes
    green_shapes = [
        *(
            dict(
                type='rect',
                xref='x',
                yref='y domain',
                x0=timestamp,
                x1=timestamp,
                y0=0,
                y1=1,
                opacity=0.5,
                line=dict(width=1, color='#30c77b'),
            )
            for timestamp, price in green_bars.items()
        ),
    ]

    red_shapes = [
        *(
            dict(
                type='rect',
                xref='x',
                yref='y domain',
                x0=timestamp,
                x1=timestamp,
                y0=0,
                y1=1,
                opacity=0.5,
                line=dict(width=1, color='#c43338'),
            )
            for timestamp, price in red_bars.items()
        ),
    ]

    # Combine all traces and create a plot
    layout = go.Layout(
        shapes=green_shapes + red_shapes,
        title=dict(text='Price with Buy/Sell Labels', font=dict(color='white')),
        xaxis=dict(
            title='Timestamp',
            gridcolor='rgba(128, 128, 128, 0.2)',
            tickfont=dict(color='white'),
            titlefont=dict(color='white'),
        ),
        yaxis=dict(
            title='Price',
            gridcolor='rgba(128, 128, 128, 0.2)',
            tickfont=dict(color='white'),
            titlefont=dict(color='white'),
        ),
        plot_bgcolor='rgba(32, 32, 32, 1)',
        paper_bgcolor='rgba(32, 32, 32, 1)',
        legend=dict(font=dict(color='white')),
        width=1100,
        height=650,
    )

    fig = go.Figure(data=[price_trace], layout=layout)
    fig.show()


In [8]:
plot_price_with_labels(data)

In [20]:
extrema.labels.value_counts(normalize=True)

 0    0.991803
 1    0.004471
-1    0.003726
Name: (0.05, 0.05, close), dtype: float64

In [23]:
def bolster_extrema(
    prices: pd.Series, extrema: pd.Series, n_pct: float
) -> pd.Series:
    extrema_copy = extrema.copy()

    for timestamp, value in extrema.items():
        if value == 0:
            continue

        price = prices[timestamp]
        lower_bound = price * (1 - n_pct)
        upper_bound = price * (1 + n_pct)

        left_timestamp = timestamp
        while True:
            left_timestamp = prices.index.get_loc(left_timestamp) - 1
            if left_timestamp < 0:
                break
            left_timestamp = prices.index[left_timestamp]

            if lower_bound <= prices[left_timestamp] <= upper_bound:
                extrema_copy[left_timestamp] = value
            else:
                break

        right_timestamp = timestamp
        while True:
            right_timestamp = prices.index.get_loc(right_timestamp) + 1
            if right_timestamp >= len(prices):
                break
            right_timestamp = prices.index[right_timestamp]

            if lower_bound <= prices[right_timestamp] <= upper_bound:
                extrema_copy[right_timestamp] = value
            else:
                break

    return extrema_copy

mod_extrema = extrema.copy()

mod_extrema.__class__.labels = property(
    mod_extrema.__class__.labels.fget,
    lambda self, value: setattr(self, '_labels', value),
)

mod_extrema.labels = bolster_extrema(data.close, extrema.labels, 0.015)

In [28]:
plot(data, mod_extrema.labels)

In [35]:
data.close.vbt.plot().show()

In [62]:
mod_extrema.labels.value_counts(normalize=True)

 0    0.861866
 1    0.070763
-1    0.067371
Name: (0.05, 0.05, close), dtype: float64

In [12]:
entries = labels == -1
exits = labels == 1

pf = vbt.PF.from_signals(data.close, entries, exits, freq='1h')

print(pf.stats())

Start                               2023-01-01 00:00:00
End                                 2023-03-17 16:00:00
Period                                 75 days 17:00:00
Start Value                                       100.0
Min Value                                     99.794558
Max Value                                  86190.717106
End Value                                  86171.469514
Total Return [%]                           86071.469514
Benchmark Return [%]                         124.332314
Total Time Exposure [%]                         53.3847
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                               1.977945
Max Drawdown Duration                   0 days 12:00:00
Total Orders                                        221
Total Fees Paid                                     0.0
Total Trades                                        111
Win Rate [%]                                      100.0
Best Trade [%]                                26